In [1]:
from decimal import Decimal
import json
import numpy as np
import psycopg2

mid_lat, mid_long = 36.00192134954784, -78.937821464194
ANG_SEP = 0.8  # angular spread (angular width) of the wedge

def rows_to_dict(rowsss, cccursor):
    columns = [desc[0] for desc in cccursor.description]
    result = []
    for rrr in rowsss:
        result.append(dict(zip(columns, rrr)))
    return result


def combine_dicts_to_dict_of_lists(dict_list):
    result = {}
    for dictionary in dict_list:
        for key, value in dictionary.items():
            if key not in result:
                result[key] = []
            result[key].append(value)
    for key in result.keys():
        if isinstance(result[key][0], Decimal):
            for iii, item in enumerate(result[key]):
                result[key][iii] = float(item)
    return result


def get_data_dict(ang_sep, uid_lower_bound=0):
    connection = psycopg2.connect(database=db_name, user=db_user,
                                  password=db_password, host=db_host, port=db_port)
    cursor = connection.cursor()
    cursor.execute("SELECT latitude as latitude, longitude as longitude, "
                   "rssi as rssi, azimuth as azimuth FROM \"SigCapDetails\" "
                   "WHERE \"mPci\"=20 and \"uid\" > " + str(int(uid_lower_bound)))
    rows = cursor.fetchall()

    df_ret = combine_dicts_to_dict_of_lists(rows_to_dict(rows, cursor))
    df_ret['azimuth_start'] = np.pi/2 - (np.radians(df_ret['azimuth']) + ang_sep/2)
    df_ret['azimuth_end'] = np.pi/2 - (np.radians(df_ret['azimuth']) - ang_sep/2)
    cursor.close()
    connection.close()
    return df_ret


with open("SQLpwd.json", 'r') as file:
    credential = json.load(file)
db_host = credential['db_host']
db_port = credential['db_port']
db_name = credential['db_name']
db_user = credential['db_user']
db_password = credential['db_password']

df_SigCapDetails = get_data_dict(ang_sep=ANG_SEP)
df_SigCapDetails['azimuth'][0:5]

[155.26122375971022,
 151.47012726089466,
 149.67721626414465,
 149.57122538860438,
 154.97488856932495]

In [2]:
from dotenv import load_dotenv
import os
import pandas as pd

load_dotenv()
API_KEY = os.getenv("API_KEY")

In [15]:
from bokeh.io import output_notebook, push_notebook, show
from bokeh.plotting import gmap#, show
from bokeh.models import ColumnDataSource, LinearColorMapper, HoverTool, GMapOptions, WheelZoomTool
from bokeh.models import ColorBar
import bokeh.palettes as palettes

output_notebook()
data_structure = {'latitude': [], 'longitude': [],
                       'rssi': [], 'azimuth': [],
                       'azimuth_start': [],
                       'azimuth_end': []}


Loading BokehJS ...

In [16]:
def plot_gmap_Duke_ss_arrow_head(inner_source, inner_field='rssi', title='rssi',
                                 mid_lat=mid_lat, mid_long=mid_long, zoom=18,
                                 inner_radius=0, outer_radius=10, alpha=0.8):
    inner_gmap_options = GMapOptions(
    lat=mid_lat,
    lng=mid_long,
    map_type='satellite',
    zoom=zoom
    )
    inner_hover = HoverTool(tooltips=[(inner_field, '@' + inner_field + '{0.0}'),
                                      #('uid', '@' + 'uid' + '{0.0}')
                                     ])

    inner_p = gmap(API_KEY, inner_gmap_options, title=title, width=600, height=450,
                   tools=[inner_hover, 'pan'])

    inner_color_mapper = LinearColorMapper(palette=palettes.inferno(20))

    inner_p.annular_wedge(x='longitude', y='latitude', inner_radius=inner_radius,
                          outer_radius=outer_radius, start_angle='azimuth_start',
                          end_angle='azimuth_end', source=inner_source,
                          color={'field': inner_field, 'transform': inner_color_mapper},
                          alpha=alpha)
    inner_color_bar = ColorBar(color_mapper=inner_color_mapper, location=(0, 0))
    inner_p.add_layout(inner_color_bar, 'right')
    inner_p.add_tools(WheelZoomTool())
    return inner_p, inner_radius, outer_radius, inner_source, inner_field, inner_color_mapper, alpha



In [21]:
import time

DELAY = 4

column_source = ColumnDataSource(data=data_structure)  # empty ColumnDataSource
# column_source = ColumnDataSource(data=get_data_dict(ang_sep=ANG_SEP))
pplot, inner_radius_, outer_radius_, inner_source_, inner_field_, inner_color_mapper_, alpha_ = \
    plot_gmap_Duke_ss_arrow_head(column_source, inner_field='rssi', title='rssi',
                                 mid_lat=mid_lat, mid_long=mid_long, zoom=18,
                                 inner_radius=0, outer_radius=10, alpha=0.8)
hand = show(pplot, notebook_handle=True)


def updateData_and_plot(handle, uid_lower_bound, gmap_plot=pplot):
    dat_struct = ColumnDataSource(data=get_data_dict(ang_sep=ANG_SEP, uid_lower_bound=uid_lower_bound))

    assert sorted(dat_struct.data.keys()) == sorted(dat_struct.data.keys()), \
        print(updateData_and_plot.__name__ + ': missing/extra dict fields')
    # both must be dicts
    for key in dat_struct.data.keys():
        assert len(dat_struct.data[key]) != 0, \
            print(updateData_and_plot.__name__ + ': empty dict value')
    gmap_plot.renderers[0].data_source.data = get_data_dict(ang_sep=ANG_SEP, uid_lower_bound=uid_lower_bound)
    push_notebook(handle=handle)
    return

In [22]:
time.sleep(3)
wedge_args = dict(x='longitude', y='latitude', inner_radius=inner_radius_,
                          outer_radius=outer_radius_, start_angle='azimuth_start',
                          end_angle='azimuth_end', source=inner_source_,
                          color={'field': inner_field_, 'transform': inner_color_mapper_},
                          alpha=alpha_)

i = 2666
while i >= 0:
    updateData_and_plot(handle=hand, gmap_plot=pplot,
                        uid_lower_bound=i)
    i = i - 10
    print(i)
    time.sleep(DELAY)

2656
2646
2636
2626
2616
2606
2596
2586
2576
2566
2556
2546
2536


KeyboardInterrupt: 